# Proyecto: Descifrando Mensajes Codificados Usando MCMC

Integrantes: Sebastián Flores y Matías Neto

## Procesamiento de la fuente de texto

In [2]:
%pip install pycipher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycipher: filename=pycipher-0.5.2-py3-none-any.whl size=30456 sha256=9de64c40398958bad39dd55ec22b4d2f7946c1fe5d214d3fe7fe9420b2d0b1ac
  Stored in directory: /home/sflores/.cache/pip/wheels/ca/e7/37/bf758675337f9b98f096d8f7a5fd0cf320aadd67ae8a12f545
Successfully built pycipher
You should consider upgrading via the '/home/sflores/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import pandas as pd
import numpy as np
from pycipher import SimpleSubstitution
import random

In [2]:
# Cargar el texto
with open("gloria_benitoperezgaldos.txt", "r") as f:
    text = f.readlines()

for i in range(len(text)):
    text[i] = text[i].replace('\n', '')

text = ' '.join(text)

In [3]:
# Sólo minúsculas
clean_text = text.casefold()

# Eliminar tildes
tilde_dict = {'á':'a', 'é': 'e', 'í': 'i', 'ó':'o', 'ú':'u', 'ñ': 'n'}
for key, value in tilde_dict.items():
    clean_text = clean_text.replace(key, value)

# Eliminar caracteres especiales
clean_text = re.sub(r"[^a-zA-Z ]", "", clean_text)
clean_text = clean_text.upper()

In [4]:
def process(word, matrix):
    '''
    Almacena la frecuencia de transiciones de una palabra en una matriz
    
    Argumentos:
        word (string): Palabra a procesar
        matrix (pd.DataFrame): Matriz de transición
    
    Retorna:
        matrix (pd.DataFrame): Matriz de transición actualizada con
                               la palabra procesada
    '''
    letters = list(word)
    matrix.loc[' ', letters[0]] = matrix.loc[' ', letters[0]] + 1
    for i in range(len(word)-1):
        matrix.loc[letters[i], letters[i+1]] = matrix.loc[letters[i], letters[i+1]] + 1
    matrix.loc[letters[len(word)-1], ' '] = matrix.loc[letters[len(word)-1], ' '] + 1
    return matrix

In [5]:
# Creación de la matriz
alphabet = sorted(''.join(set(clean_text)))
matrix = pd.DataFrame(data=np.zeros((len(alphabet), len(alphabet))), index=alphabet, columns=alphabet)
words = clean_text.split()

# Se demora unos minutos
for word in words:
    matrix = process(word, matrix)
transition = matrix.apply(lambda x: x/np.sum(x), axis=1)

## Definición de funciones auxiliares para la minimización

In [34]:
def plausibility(ciphertext, key, matrix = transition):
    '''
    Función de plausibilidad que tiene el rol de -U para SA
    
    Argumentos:
        ciphertext (string): Texto codificado
        key (string): Clave del mismo largo que el alfabeto a testear
        matrix (pd.DataFrame): Matriz de transición construida antes
    
    Retorna:
        plausibility (float): valor de la plausibilidad para key
    '''
    cipher = SimpleSubstitution(key)
    plaintext = cipher.decipher(ciphertext, keep_punct = True)
    words = plaintext.split()
    matrix = pd.DataFrame(data=np.zeros((len(alphabet), len(alphabet))), index=alphabet, columns=alphabet)
    for word in words:
        matrix = process(word, matrix)
    plausibility = np.sum(matrix.to_numpy() * np.log(1+transition.to_numpy()))
    return plausibility

# Ejemplo
plaintext = "MUCHOS ANOS DESPUES FRENTE AL PELOTON DE FUSILAMIENTO " + \
            "EL CORONEL AURELIANO BUENDIA RECORDO AQUELLA TARDE REMOTA " + \
            "EN QUE SU PADRE LO LLEVO A CONOCER EL HIELO"
real_key = "AJPCZWRLFBDKOTYUQGENHXMIVS"
real_cipher = SimpleSubstitution(real_key)
ciphertext = real_cipher.encipher(plaintext, keep_punct = True)
print(f"Texto cifrado: {ciphertext}")
print(f"Plausibilidad: {plausibility(ciphertext, real_key)}")

Texto cifrado: OHPLYE ATYE CZEUHZE WGZTNZ AK UZKYNYT CZ WHEFKAOFZTNY ZK PYGYTZK AHGZKFATY JHZTCFA GZPYGCY AQHZKKA NAGCZ GZOYNA ZT QHZ EH UACGZ KY KKZXY A PYTYPZG ZK LFZKY
Plausibilidad: 23.15141138155179


In [35]:
# Ejemplo para ver que efectivamente la plausibilidad es menor
example_key = ''.join(random.sample(real_key, len(real_key)))
print("Clave ejemplo: " + example_key)
example_cipher = SimpleSubstitution(example_key)
print("Texto descifrado con la clave: "+example_cipher.decipher(ciphertext, keep_punct=True))
print(f"Plausibilidad: {plausibility(ciphertext, example_key)}")

Clave ejemplo: VBXMPQYGFDAJKUTHELCRONWZSI
Texto descifrado con la clave: UPERGQ KOGQ SXQNPXQ WHXOVX KM NXMGVGO SX WPQIMKUIXOVG XM EGHGOXM KPHXMIKOG LPXOSIK HXEGHSG KFPXMMK VKHSX HXUGVK XO FPX QP NKSHX MG MMXCG K EGOGEXH XM RIXMG
Plausibilidad: 5.819094712134424


In [36]:
def neighbor(key):
    '''
    Retorna un vecino aleatorio (con distribución uniforme) de la clave
    
    Argumentos:
        key (str): Clave representada por una permutación del alfabeto
    
    Retorna:
        new_key (str): Clave vecina a key
    '''
    indices = random.sample(list(np.arange(len(key))), 2)
    symbol_a, symbol_b = key[indices[0]], key[indices[1]]
    new_key = list(key)
    new_key[indices[0]] = symbol_b
    new_key[indices[1]] = symbol_a
    new_key = ''.join(new_key)
    return new_key

nghbr = neighbor(example_key)
print(f"Clave ejemplo: {example_key}")
print(f"Vecino: {nghbr}")
print(f"Indices intercambiados: {[i for i in range(len(example_key)) if example_key[i] != nghbr[i]]}")

Clave ejemplo: VBXMPQYGFDAJKUTHELCRONWZSI
Vecino: ABXMPQYGFDVJKUTHELCRONWZSI
Indices intercambiados: [0, 10]


In [37]:
''.join(sorted(list(real_key)))

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

## Optimización

In [38]:
def SADecipher(ciphertext: str, Niter: int, beta,
               initial_key = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
               print_period = None):
    '''
    Función que implementa Simmulated Annealing 
    para el problema de descifrado
    
    Argumentos:
        ciphertext: Código a descifrar
        Niter: Número de pasos a realizar
        beta (fun): Función que representa el beta del método de SA
        initial_key (str): una clave inicial para el algoritmo
        print_period: Representa el periodo de actualización. 
            Si no es None, se imprime el estado de la cadena
            y la plausibilidad cada print_period pasos
    
    Retorna:
        key: Estado de la cadena luego de Niter pasos
    '''
    
    U = np.random.uniform(0, 1, size = Niter)
    key = initial_key
    incumbent_plaus = plausibility(ciphertext, key)
    if print_period == None:
        for i in range(Niter):
            new_key = neighbor(key)
            new_plaus = plausibility(ciphertext, new_key)
            if U[i] <= np.exp(beta(i) * (new_plaus - incumbent_plaus)):
                key = new_key
                incumbent_plaus = new_plaus
    else:
        for i in range(Niter):
            new_key = neighbor(key)
            new_plaus = plausibility(ciphertext, new_key)
            if U[i] <= np.exp(beta(i) * (new_plaus - incumbent_plaus)):
                key = new_key
                incumbent_plaus = new_plaus
            if i % print_period == 0:
                cipher = SimpleSubstitution(key)
                print(f"Paso {i}")
                print(f"Frase actual: {cipher.decipher(ciphertext, keep_punct = True)}")
                print(f"Plausibilidad: {incumbent_plaus}")
    return key

# Test de SADecipher
beta = lambda x: 1     
sample_advance = SADecipher(ciphertext, 10, beta)
print(f"Estado de la cadena luego de 10 pasos con beta = 1: {sample_advance}")
print(f"Indices intercambiados: {[i for i in range(len(example_key)) if 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[i] != sample_advance[i]]}")

Estado de la cadena luego de 10 pasos con beta = 1: UBCIGFAWNJKLMDOHPQSTEVRXYZ
Indices intercambiados: [0, 3, 4, 6, 7, 8, 13, 15, 16, 17, 20, 22]


In [40]:
beta = lambda x: 10**(-3) * x
sample_advance = SADecipher(ciphertext, 10000, beta, print_period = 1000)
print(f"Estado de la cadena luego de 10 pasos con beta lineal: {sample_advance}")
print(f"Indices intercambiados: {[i for i in range(len(example_key)) if 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[i] != sample_advance[i]]}")

Paso 0
Frase actual: OHPLYE ATYE CZEUHZE WGZTDZ AK UZKYDYT CZ WHEFKAOFZTDY ZK PYGYTZK AHGZKFATY JHZTCFA GZPYGCY AQHZKKA DAGCZ GZOYDA ZT QHZ EH UACGZ KY KKZXY A PYTYPZG ZK LFZKY
Plausibilidad: 11.195625132491456
Paso 1000
Frase actual: MUKLON YPON DENSUEN WAEPVE YC SECOVOP DE WUNTCYMTEPVO EC KOAOPEC YUAECTYPO BUEPDTY AEKOADO YRUECCY VYADE AEMOVY EP RUE NU SYDAE CO CCEFO Y KOPOKEA EC LTECO
Plausibilidad: 15.746106594462622
Paso 2000
Frase actual: RPGQYT ZBYT KATWPAT UDABIA ZL WALYIYB KA UPTJLZRJABIY AL GYDYBAL ZPDALJZBY VPABKJZ DAGYDKY ZNPALLZ IZDKA DARYIZ AB NPA TP WZKDA LY LLAOY Z GYBYGAD AL QJALY
Plausibilidad: 11.516343036029205
Paso 3000
Frase actual: WITZEK ONEK FAKSIAK VHANJA OL SALEJEN FA VIKMLOWMANJE AL TEHENAL OIHALMONE QIANFMO HATEHFE OCIALLO JOHFA HAWEJO AN CIA KI SOFHA LE LLADE O TENETAH AL ZMALE
Plausibilidad: 19.372825902183585
Paso 4000
Frase actual: QZJYOK ESOK RAKVZAK DHASTA EN VANOTOS RA DZKINEQIASTO AN JOHOSAN EZHANIESO MZASRIE HAJOHRO EXZANNE TEHRA HAQOTE AS XZA KZ V